Morris-Lecar Stochastic Square-Wave Burster
===========================================

Here we go through the necessary definitions and computations for the bursting models.|

I exclude all the visualisations because those were subject to change as per requirements and i was just too lazy to come up with a way to organise that differently for square-wave and elliptic. 

Sometime in the future it will be merged.



import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D 
import sys

In [2]:
C_m = 20.0
V_Ca = 120.0
V_K = -84.0
V_L = -60.0
g_Ca = 4.0   # Calcium conductance
g_K = 8.0    # Potassium conductance
g_L = 2.0    # Leak conductance
g_z = 1.0    # Slow adaptation conductance
V_z = -84.0  # Reversal potential for slow current

# Gating parameters
v1, v2 = -1.2, 18.0
v3, v4 = 12.0, 17.4

'''differencial parameterisation'''
phi = 0.0667

# Slow variable parameters
epsilon_default = 0.005 
noise_default = 0.00  
I_app_default = 45.0

In [5]:
#For elliptic we have this extra piece-

REGIMES = {
    'Soft (Supercritical)': {
        'desc': "Soft Onset: Oscillations grow from zero amplitude.",
        'v1': -1.2, 'v2': 18.0, 
        'v3': 2.0,  'v4': 30.0, 
        'g_Ca': 4.4, 'phi': 0.04,
        'I_target': 100.0,
        'eps': 0.002
    },
    'Hard (Subcritical)': {
        'desc': "Hard Onset: Jumps to large amplitude immediately.",
        'v1': -1.2, 'v2': 18.0, 
        'v3': 10.0, 'v4': 15.0, 
        'g_Ca': 4.0, 'phi': 0.067,
        'I_target': 90.0,
        'eps': 0.002
    }
}

P = REGIMES['Soft (Supercritical)'].copy()
P['noise'] = 0.05

In [3]:

def m_inf(V): return 0.5 * (1 + np.tanh((V - v1) / v2))
def w_inf(V): return 0.5 * (1 + np.tanh((V - v3) / v4))
def tau_w(V): return 1.0 / np.cosh((V - v3) / (2 * v4))

In [ ]:

def get_z_nullcline_v(I_app):
    """Calculates the Z-curve (V-z Nullcline) for a given I_app."""
    Vs = np.linspace(-75, 50, 500)
    w_ss = w_inf(Vs)
    i_fast = g_Ca*m_inf(Vs)*(Vs-V_Ca) + g_K*w_ss*(Vs-V_K) + g_L*(Vs-V_L)
    den = g_z * (Vs - V_K)
    den[np.abs(den) < 1e-6] = 1e-6
    Zs = (I_app - i_fast) / den
    return Vs, Zs

def get_v_nullcline_w(Vs, current_z, I_app):
    """Calculates V-nullcline in V-w plane."""
    m = m_inf(Vs)
    num = I_app - g_Ca*m*(Vs-V_Ca) - g_L*(Vs-V_L) - g_z*current_z*(Vs-V_K)
    den = g_K * (Vs - V_K)
    den[np.abs(den) < 1e-6] = 1e-6
    return num / den

def sde_step(state, dt, I_base, eps, noise_str):
    V, w, z = state
    
    # Fast Variables
    m = m_inf(V)
    w_ss = w_inf(V)
    tau = tau_w(V)
    
    i_sum = (g_Ca * m * (V - V_Ca) + 
             g_K * w * (V - V_K) + 
             g_L * (V - V_L) + 
             g_z * z * (V - V_K))
    
    dV = (I_base - i_sum) / C_m * dt
    dw = phi * (w_ss - w) / tau * dt
    
    # Slow Variable (Stochastic)
    z_inf = 0.5 * (1 + np.tanh((V - (-20)) / 10)) 
    dW = np.random.normal(0, np.sqrt(dt)) * noise_str
    dz = eps * (z_inf - z) * dt + dW
    
    return [V + dV, w + dw, z + dz]